블로그 리뷰 10개 샘플 가지고 시험

In [1]:
from tqdm.notebook import tqdm
import re
from collections import Counter
import math

with open('./data/lib1.txt', 'r', encoding='utf-8') as f:
    txt = f.read()

In [2]:
# remove tab
txt_clean = re.sub(r'\s+', ' ', txt).strip()
# remove NBSP
txt_clean = txt_clean.replace('\xa0', ' ')
# remove invis char
txt_clean = txt_clean.replace('\u200b', '')
# remove ZWSP
txt_clean = re.sub(r'[\u200B]', '', txt_clean)
# remove html elements
txt_clean = re.sub(r'<.*?>', '', txt_clean)
# print(txt_clean)

In [3]:
from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis",
    model="WhitePeak/bert-base-cased-Korean-sentiment",
)

2025-06-16 12:48:20.067539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750045700.122689  242188 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750045700.138069  242188 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750045700.257910  242188 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750045700.257930  242188 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750045700.257931  242188 computation_placer.cc:177] computation placer alr

In [4]:
import spacy

# small model
nlp = spacy.load("ko_core_news_sm")
# 데탑용 large model (nvidia-smi)
# nlp = spacy.load("ko_core_news_lg")

nlp.add_pipe("sentencizer")  # rule-based

In [5]:
doc = nlp(txt_clean)

In [6]:
[print(l) for l in [sent.text for sent in doc.sents][:10]]
print()

네이버 강남역과 역삼역 사이에 위치하고 있는 역삼도서관에 다녀왔습니다.
 
표지판에 역삼 도서관이 적혀 있어서 다 왔구나!
했는데..  ??
역삼 1동 주민센터와 문화센터만 보여서 당황했습니다..  가까이 가보면 왼쪽 한켠에 ‘강남구청 역삼도서관’이라는 작은 간판이 있습니다.  
들어가자마자 엘레베이터를 타고 5층으로 가면 역삼도서관이 있습니다.  
구립 도서관이다보니 5층 한층만 도서관이에요.  
자료실 화-금 09:00-22:00 / 주말 09:00-17:00 멀티미디어실 화-금 09:00-18:00 / 주말 09:00-17:00 일반열람실 화-일 07:00-22:00  휴관일  매주 월요일 및 법정 공휴일     멀티미디어실 멀티미디어실은 좌석을 예약한 뒤 PC는 1일 최대 3시간 노트북 좌석은 최대 5시간까지 이용가능합니다.  
열람실 보통 구립 도서관은 열람실도 9 to 6인 경우가 많은데 여기는 7시부터 22시까지 운영합니다.    
도서관 입구에 있는 키오스크에서 좌석 배정 후 이용 가능해요.  



In [7]:
# Apply to a list of reviews
doc_sents = [sent.text if len(sent.text) < 512 else sent.text[:512] for sent in doc.sents]
sentiment_cat = [('positive' if classifier(sent)[0]['label'] == 'LABEL_1' else 'negative', sent)
                 for sent in tqdm(doc_sents)]

  0%|          | 0/269 [00:00<?, ?it/s]

블로그 내용 자체의 연과성과 문장 분리부터 좀 불안한 관계로 sentiment analysis또한 일관적이지가 않은 편임.
sentiment analysis 사용 안하기로 판정

In [8]:
sentiment_cat[:10] , sentiment_cat[10:]

([('positive', '네이버 강남역과 역삼역 사이에 위치하고 있는 역삼도서관에 다녀왔습니다.'),
  ('negative', ' '),
  ('negative', '표지판에 역삼 도서관이 적혀 있어서 다 왔구나!'),
  ('negative', '했는데..  ??'),
  ('negative',
   '역삼 1동 주민센터와 문화센터만 보여서 당황했습니다..  가까이 가보면 왼쪽 한켠에 ‘강남구청 역삼도서관’이라는 작은 간판이 있습니다.  '),
  ('negative', '들어가자마자 엘레베이터를 타고 5층으로 가면 역삼도서관이 있습니다.  '),
  ('negative', '구립 도서관이다보니 5층 한층만 도서관이에요.  '),
  ('positive',
   '자료실 화-금 09:00-22:00 / 주말 09:00-17:00 멀티미디어실 화-금 09:00-18:00 / 주말 09:00-17:00 일반열람실 화-일 07:00-22:00  휴관일  매주 월요일 및 법정 공휴일     멀티미디어실 멀티미디어실은 좌석을 예약한 뒤 PC는 1일 최대 3시간 노트북 좌석은 최대 5시간까지 이용가능합니다.  '),
  ('positive',
   '열람실 보통 구립 도서관은 열람실도 9 to 6인 경우가 많은데 여기는 7시부터 22시까지 운영합니다.    '),
  ('positive', '도서관 입구에 있는 키오스크에서 좌석 배정 후 이용 가능해요.  ')],
 [('positive', '강남역 1번 출구에서도 멀지 않은 곳에 위치하고 있어 시간이 붕 떴을 때 잠시 들리기도 좋구..  '),
  ('positive', '열람실도 깔끔하게 잘 되어 있어 공부하러 오기도 좋을 것 같 습니다.      '),
  ('positive', '[STOP] 네이버 주말에 서울 강남구 역삼 도서관 에 방문하였습니다!'),
  ('positive', '(역삼역, 강남역 근처)  건물 옆에 공영주차장 도 있습니다!  '),
  ('positive', '* 위치 : 역삼1동 주민

---

단순 count기반 키워드 추출

In [9]:
from konlpy.tag import Okt
okt = Okt()

def preprocess_txt(corpus: str):
    # norm=True applies orthographic normalization
    # stem=True reduces to word stem
    lemmas = [(token, pos) for token, pos in okt.pos(corpus, norm=True, stem=True)]
    # 특수문자 제거
    lemmas_clean = [(re.sub(r'[^가-힣a-zA-Z\s]', '', t), p) for t, p in lemmas]
    # 한개짜리 단어 제거거
    lemmas_clean = [(t, p) for t, p in lemmas_clean if len(t) > 1]

    return lemmas_clean

In [10]:
lemmas_clean = preprocess_txt(txt_clean)
lemmas_clean[:25]

[('네이버', 'Noun'),
 ('강남역', 'Noun'),
 ('역삼역', 'Noun'),
 ('사이', 'Noun'),
 ('위치', 'Noun'),
 ('하고', 'Josa'),
 ('있다', 'Adjective'),
 ('역삼', 'Noun'),
 ('도서관', 'Noun'),
 ('다녀오다', 'Verb'),
 ('표지판', 'Noun'),
 ('역삼', 'Noun'),
 ('도서관', 'Noun'),
 ('적히다', 'Verb'),
 ('있다', 'Adjective'),
 ('오다', 'Verb'),
 ('하다', 'Verb'),
 ('역삼', 'Noun'),
 ('주민', 'Noun'),
 ('센터', 'Noun'),
 ('문화센터', 'Noun'),
 ('여서', 'Josa'),
 ('당황', 'Noun'),
 ('하다', 'Verb'),
 ('가까이', 'Noun')]

In [11]:
poses = Counter([l[1] for l in lemmas_clean])
poses

Counter({'Noun': 2020,
         'Verb': 601,
         'Adjective': 306,
         'Josa': 172,
         'Alpha': 94,
         'Adverb': 56,
         'Hashtag': 55,
         'URL': 44,
         'Foreign': 17,
         'Modifier': 17,
         'Number': 15,
         'Conjunction': 13,
         'Eomi': 2,
         'ScreenName': 2})

In [12]:
use_pos = [  # 문장성분 추려내기
    'Noun',
    'Verb',
    'Adjective',
    # 'Adverb',
    'Hashtag',
]
lemmas_clean_fin = [kw for kw, pos in lemmas_clean
                    if pos in use_pos]
len(lemmas_clean_fin)

2982

In [13]:
word_list = [(*okt.pos(w)[0], c)
             for w, c in Counter(lemmas_clean_fin).items()
             if c > 1]  # 너무 적은 것들은 포함 안함.

[(*okt.pos(w)[0], c)
 for w, c in Counter(lemmas_clean_fin).most_common(25)]

[('하다', 'Verb', 146),
 ('도서관', 'Noun', 105),
 ('있다', 'Adjective', 94),
 ('역삼', 'Noun', 55),
 ('좌석', 'Noun', 35),
 ('이용', 'Noun', 31),
 ('되다', 'Verb', 26),
 ('보다', 'Verb', 25),
 ('시간', 'Noun', 24),
 ('열람', 'Noun', 22),
 ('가다', 'Verb', 21),
 ('좋다', 'Adjective', 20),
 ('공부', 'Noun', 20),
 ('공연', 'Noun', 20),
 ('예약', 'Noun', 18),
 ('이다', 'Josa', 18),
 ('센터', 'Noun', 16),
 ('같다', 'Adjective', 16),
 ('주민', 'Noun', 15),
 ('노트북', 'Noun', 15),
 ('회원', 'Noun', 15),
 ('들다', 'Verb', 15),
 ('강남역', 'Noun', 14),
 ('강남구', 'Noun', 14),
 ('읽다', 'Verb', 14)]

TF-IDF통해서 키워드 추출

블로그 하나를 document로 정의

블로그별 top n단어 추출해서 집합하기?

```python
target = '목표 단어'
TF = Counter(blogs['blog A'])['target']  # target count for blog A
IDF = log(len(blogs) / (1 + len([blog for blog in blogs if target in blog])))

# tf-idf for blog A is:
TF * TDF
```

나중에는 집합한 TF-IDF들에 대하여 또 TF-IDF를 계산해서 도서관별 돋보이는 keyword 찾을수도? (충분히 vary한다면 될 듯)

In [14]:
# 그동안 한것들 pipeline으로
def preprocess_txt_ex(corpus: str):
    lemmas = [(token, pos) for token, pos in okt.pos(corpus, norm=True, stem=True)]
    lemmas_clean = [(re.sub(r'[^가-힣a-zA-Z\s]', '', t), p) for t, p in lemmas]
    lemmas_clean = [(t, p) for t, p in lemmas_clean if len(t) > 1]

    use_pos = [
        'Noun',
        'Verb',
        'Adjective',
        'Hashtag',
    ]
    lemmas_clean = [kw for kw, pos in lemmas_clean
                    if pos in use_pos]

    word_counts = {w: c
                   for w, c in Counter(lemmas_clean).items()
                   if c > 1}

    return word_counts

In [15]:
# TF-IDF를 위한 블로그별 그룹화
txt_clean_grouped = txt_clean.split('[STOP]')[:-1]  # [STOP] 기준 블로그 delimeter 쪼개기 후, 마지막 꼬투리 제거
blogs_clean_grouped = [preprocess_txt_ex(c) for c in txt_clean_grouped]
print('no. group:', len(blogs_clean_grouped))

no. group: 10


In [16]:
# 위에꺼들 pipeline으로 만들기
# 블로그별 상위 n개만 저장
_word_list = list(set(  # 중복제거
    [k for blog in blogs_clean_grouped for k, _ in blog.items()]
))

df_scores = [(w, len([True for blog in blogs_clean_grouped if w in blog])) for w in _word_list]

doc_len = len(blogs_clean_grouped)
idf_scores = [(w, math.log(doc_len / (1 + df)) ) for w, df in df_scores]

total_tf_idf_top = list()
for blog in blogs_clean_grouped:  # 각 document(blog)에 대한 TF-IDF점수를 구할 수 있음.
    tf_scores = [(w, blog.get(w, 0)) for w in _word_list]
    tfidf_scores = [(tf[0], tf[1] * idf[1]) for tf, idf in zip(tf_scores, idf_scores) if tf[1] > 0]

    tfidf_scores_top = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:10]  # top 10
    total_tf_idf_top += tfidf_scores_top

total_tf_idf_top.sort(key=lambda x: x[1], reverse=True)
[print(l) for l in total_tf_idf_top[:25]]
print()

('공연', 20.467537673540914)
('노트북', 19.313254949209202)
('쓰다', 17.703817036775103)
('홈페이지', 14.484941211906902)
('예약', 12.03972804325936)
('강의', 11.266065387038703)
('글쓰기', 11.266065387038703)
('경선', 11.266065387038703)
('여행', 11.266065387038703)
('클래식', 11.266065387038703)
('좌석', 11.090354888959125)
('작가', 9.656627474604601)
('여름', 9.656627474604601)
('코로나', 9.656627474604601)
('선택', 9.656627474604601)
('여중', 9.656627474604601)
('태안', 9.656627474604601)
('예약', 8.427809630281553)
('읽다', 8.427809630281553)
('들다', 8.246616586867397)
('좌석', 7.6246189861593985)
('학습', 7.223836825955617)
('집중', 7.223836825955617)
('입구', 6.437751649736401)
('이름', 6.437751649736401)

